# Initial optical alignment procedure.

This notebook implementats the initial optical alignment procedure for the Main Telescope.

This procedure consist in using the Laser Tracker (MTAlignment) component to measure the position of M2 and Camera with respect to M1M3 and offseting them such that they are as close to alignment as possible before going on sky.

This implementation considers that the MTCS components are ready and in position before starting, e.g., it will not move the telescope and/or do any additional verification that they are ready.
We may want to improve this in a later stage.


In [ ]:
import asyncio

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS

## Startup communication with the MTAlignment and MTCS

In [ ]:
domain = salobj.Domain()

In [ ]:
mtalign = salobj.Remote(domain, "MTAlignment")

In [ ]:
await mtalign.start_task

In [ ]:
mtcs = MTCS(domain)

In [ ]:
await mtcs.start_task

In [ ]:
mtcs.set_rem_loglevel(100)

## Check liveliness of the system

In [ ]:
await mtalign.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await mtcs.assert_liveliness()

## Setup the laser tracker

On the current version of the system (xml 12.0.0), the MTAlignment CSC has no way to report the state of the laser tracker.
In general we would want to check that the laser is powered on, using an event, and then proceed to power it on if needed.
Then, once powered on we would want to wait until it is ready to operate.

Because this is not possible yet we have to make sure the laser tracker is up and running "manually" before proceeding.
For that, we leave here the cells required to enabled the MTAlignment CSC, power it on than wait 60s before proceeding.

The following might fail is the laser is already on or if it takes longer than 60s to warm up, so continue with caution.


In [ ]:
await salobj.set_summary_state(mtalign, salobj.State.ENABLED)

In [ ]:
await mtalign.cmd_setLogLevel.set_start(level=20)

In [ ]:
await mtalign.cmd_laserPower.set_start(power=1)

In [ ]:
await asyncio.sleep(60.)

## Align M2

Alignment of M2 uses the M2 Hexapod.

In [ ]:
mtalign.evt_offsetsPublish.flush()
await mtalign.cmd_align.set_start(target=1, timeout=5)
m2_offset = await mtalign.evt_offsetsPublish.next(flush=False, timeout=5)

In [ ]:
print(m2_offset.dX*1e6, m2_offset.dY*1e6, m2_offset.dZ*1e6, m2_offset.dRX*3600., m2_offset.dRY*3600., m2_offset.dRZ*3600.)

In [ ]:
abs(m2_offset.dX) > 1e-7

In [ ]:
tolerance_m = 1.0e-7
tolerance_deg = 5.0/3600.0
max_iter = 10

for n_iter in range(max_iter):
    # Get M2 Hexapod offset
    print("Measure m2 hexapod alignment.")
    mtalign.evt_offsetsPublish.flush()
    await mtalign.cmd_align.set_start(target=1, timeout=5)
    m2_offset = await mtalign.evt_offsetsPublish.next(flush=False, timeout=5)
    corrections = [
        m2_offset.dX*1e6 if abs(m2_offset.dX) > tolerance_m else 0.0,
        m2_offset.dY*1e6 if abs(m2_offset.dY) > tolerance_m else 0.0,
        m2_offset.dZ*1e6 if abs(m2_offset.dZ) > tolerance_m else 0.0,
        m2_offset.dRX if abs(m2_offset.dRX) > tolerance_deg else 0.0,
        m2_offset.dRY if abs(m2_offset.dRX) > tolerance_deg else 0.0,
    ]

    if all([abs(corr) > 0.0 for corr in corrections]):
        print(f"[{n_iter+1:02d}:{max_iter:02d}]: Applying corrections: {corrections}")
        await mtcs.offset_m2_hexapod(
            x=-corrections[0],
            y=-corrections[1],
            z=-corrections[2],
            u=-corrections[3],
            v=-corrections[4],
        )
    else:
        measured_corrections = [
            m2_offset.dX*1e6,
            m2_offset.dY*1e6,
            m2_offset.dZ*1e6,
            m2_offset.dRX,
            m2_offset.dRY,
        ]        
        print(f"Correction completed: {measured_corrections}!")
        break

## Align Camera


In [ ]:
tolerance_m = 1.0e-7
tolerance_deg = 5.0/3600.0
max_iter = 10

for n_iter in range(max_iter):
    # Get Camera Hexapod offset
    print("Measure camera hexapod alignment.")
    mtalign.evt_offsetsPublish.flush()
    await mtalign.cmd_align.set_start(target=3, timeout=5)
    cam_offset = await mtalign.evt_offsetsPublish.next(flush=False, timeout=5)
    corrections = [
        cam_offset.dX*1e6 if abs(cam_offset.dX) > tolerance_m else 0.0,
        cam_offset.dY*1e6 if abs(cam_offset.dY) > tolerance_m else 0.0,
        cam_offset.dZ*1e6 if abs(cam_offset.dZ) > tolerance_m else 0.0,
        cam_offset.dRX if abs(cam_offset.dRX) > tolerance_deg else 0.0,
        cam_offset.dRY if abs(cam_offset.dRX) > tolerance_deg else 0.0,
    ]

    if all([abs(corr) > 0.0 for corr in corrections]):
        print(f"[{n_iter+1:02d}:{max_iter:02d}]: Applying corrections: {corrections}")
        await mtcs.offset_camera_hexapod(
            x=-corrections[0],
            y=-corrections[1],
            z=-corrections[2],
            u=-corrections[3],
            v=-corrections[4],
        )
    else:
        measured_corrections = [
            cam_offset.dX*1e6,
            cam_offset.dY*1e6,
            cam_offset.dZ*1e6,
            cam_offset.dRX,
            cam_offset.dRY,
        ]        
        print(f"Correction completed: {measured_corrections}!")
        break